### HMC (Hamiltonian Monte Carlo)

Metropolis-Hastings (HM)的优点：
- MH算法步骤非常精简，每次迭代iteration只有两个步骤
   1. 根据上次采样的参数生成proposal distribution，然后进行采样

   `proposal_θ1 = st.norm(proposal_θ0,0.5).rvs(1)`

   2. 计算拒绝概率(接受／拒绝)

   ```
   unstd_posterior1 = calculate_posterior(proposal_θ1, data)
   ratio = unstd_posterior1 / unstd_posterior0
   if ratio < st.uniform(0, 1).rvs(1): print("接受")
   ```

- MH算法可以从复杂的后验分布中采样，虽然其效率不高。

Metropolis-Hastings (HM)的问题：
- proposal distribution 的标准差太大会导致大量被拒绝的样本，而太小的标准差会导致花费更长时间来“探索分布”

  `proposal_θ1 = st.norm(proposal_θ0,0.5).rvs(1)` 比如将这里的0.5修改为5

- 当模型参数很多的时候，MH 采样的效率非常低，因为它的采样是根据 proposal distribution随机选取的
- 对于高维后验分布，MH 的采样是非常不均匀的


![Image Name](https://cdn.kesci.com/upload/image/rjvgyvcsre.gif?imageView2/0/w/640/h/640)


绿色为接受的采样，红色是拒绝的采样。可以看到HM算法中被拒绝的采样是比较多的。

**HMC (Hamiltonian Monte Carlo)**

为了解决 HM 中 proposal distribution 由于随机采样导致效率低的问题，HMC孕育而生。

HMC结合物理概念来优化 MCMC：
- HMC 通过基本物理规律 **能量=势能+动能** 假设采样的分布(后验分布)为系统的总能量，然后将该系统分为两个子部分
- 由于动量增加，势能就会减少，因此**系统总能量不会变化**。这说明我们采样的分布不会因为系统的动态变化而变化，这类似于马尔科夫的平稳分布


![Image Name](https://cdn.kesci.com/upload/image/rjviiji42v.gif?imageView2/0/w/640/h/640)


- 势能，与位置有关，比如对于重力势能，高度越高，势能越大。相似的，可以把参数的位置看作高度，把参数对应的似然看作势能。可以结合拒绝采样进行理解，在拒绝采样中，似然值越大对应的参数越有可能被接受，因此，这里势能越大的参数也越有可能被接受。

	```
	unstd_posterior0 = calculate_likelihood(proposal_θ0, data)  # 上一次参数对应的似然值(即势能)
	unstd_posterior1 = calculate_likelihood(proposal_θ1, data)  # 当前次参数对应的似然值
	ratio = unstd_posterior1 / unstd_posterior0                          # 与 HM一样计算两次参数似然的比值
	``` 
	
- 动能，与速度有关，比如动能公式 $\frac{1}{2}mv^2$。其对应的是似然函数的梯度，从下图中可以发现，沿着梯度进行采样可以更多的采样到分布 peak 位置部分的参数。
	
	```
	momentum0 = momentum_dist.rvs()                                                 # 随机生成动能值
	gradient = calculate_gradient(momentum0, likelihood_function)     # 根据动能值计算似然函数的梯度
	momentum1 = momentum0 + gradient                                            # 根据梯度更新动能
	
	# 根据
	momentum_ratio = calculate_likelihood(momentum0, data)/(momentum1, data)
	```


- 需要注意的是，势能会随着动能的变化而变化：

```
proposal_θ1 = unstd_posterior0 + momentum                                # 根据动能更新参数位置
unstd_posterior1 = calculate_likelihood(proposal_θ1, data)             # 根据更新的位置计算势能
```

- 最后结合两部分能量进行判断

```
if ratio*momentum_ratio < st.uniform(0, 1).rvs(1): print("接受")
```


![Image Name](https://cdn.kesci.com/upload/image/rjvh3zx4an.gif?imageView2/0/w/640/h/640)


绿色为接受的采样，红色是拒绝的采样。可以看到HMC几乎没有被拒绝的采样，其采样效率高。

红色的线条是 HMC 表示沿着与梯度有关的 proposal distritbution 进行采样。

HMC 与 HM 相比的**主要区别**在于：
- HMC 凭借更少的迭代数来获得相对多的具有代表性的采样
- HMC 因为每次单次迭代需要计算似然函数的梯度，因此计算的“消耗”更高，但 HMC 仍然比HM效率更高
- HMC 在大多数情况下接受采样的概率比 HM 高

基于HMC，最常用的算法是 No-U-Turn sampler（NUTS），pymc 和 Stan默认使用的也是NUTS算法。


![Image Name](https://cdn.kesci.com/upload/image/rjvix0jc6b.png?imageView2/0/w/320/h/320)


![Image Name](https://cdn.kesci.com/upload/image/rjviyt1d5d.png?imageView2/0/w/320/h/320)


**吉布斯采样 (Gibbs sampling)**

Gibbs sampling 通过从参数的条件分布进行采样或从给定的另一个参数值的概率分布进行采样。

![Image Name](http://gorayni.github.io/assets/posts/gibbs/gibbs2.gif)


**差分进化算法 (Differential evolution)**

Differential evolution


![Image Name](https://matteding.github.io/images/diff_evol.gif)
